<a href="https://colab.research.google.com/github/TuragDev/Predicting-Crystal-System-with-ML-DL/blob/main/Crystal_structure_prediction_by_ANN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install scikit-optimize

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.3/100.3 kB 2.2 MB/s eta 0:00:00


In [2]:
pip install keras-tuner


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 128.9/128.9 kB 2.8 MB/s eta 0:00:00


In [3]:
!pip install scikit-learn

In [4]:
pip install keras

In [5]:
pip install tensorflow

In [6]:
!pip install scikeras
from scikeras.wrappers import KerasClassifier

In [24]:
pip install keras-tuner

In [37]:
import numpy as np
from sklearn.model_selection import train_test_split
from skopt import BayesSearchCV
from skopt.space import Real, Integer
import joblib
from sklearn.model_selection import train_test_split
import numpy as np

import pandas as pd

from sklearn.model_selection import GridSearchCV, RepeatedKFold
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error as MSE
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score

from sklearn.model_selection import train_test_split

import numpy as np

import pickle
import pandas as pd
from sklearn.preprocessing import LabelEncoder
import sys
from sklearn import svm
from sklearn.model_selection import GridSearchCV
from numpy import mean
from numpy import std
from sklearn.datasets import make_classification
from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, classification_report
from scikeras.wrappers import KerasClassifier
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.utils import to_categorical
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
from pandas import read_csv
from sklearn.model_selection import cross_val_score
from sklearn.svm import SVC
from sklearn.model_selection import RepeatedStratifiedKFold
from skopt import BayesSearchCV
from sklearn.utils import shuffle
from kerastuner.tuners import RandomSearch
from kerastuner.tuners import BayesianOptimization

In [9]:
from google.colab import files
uploaded= files.upload()

Saving Orbital_test.csv to Orbital_test.csv
Saving Orbital_train.csv to Orbital_train.csv
Saving Sine_test.csv to Sine_test.csv
Saving Sine_train.csv to Sine_train.csv
Saving Xrd_test.csv to Xrd_test.csv
Saving Xrd_train.csv to Xrd_train.csv


In [10]:
df= pd.read_csv('Sine_train.csv')
df.shape

(596, 166)

In [11]:
X_train=df.iloc[:,14:]
y_train=df['crystal_system']

y_train

0          trigonal
1        tetragonal
2      orthorhombic
3      orthorhombic
4             cubic
           ...     
591      monoclinic
592        trigonal
593    orthorhombic
594        trigonal
595      monoclinic
Name: crystal_system, Length: 596, dtype: object

In [12]:
X_train.shape

(596, 152)

In [13]:
df=pd.read_csv('Sine_test.csv')
X_test=df.iloc[:,14:]

y_test=df['crystal_system']

y_test

0           cubic
1           cubic
2       triclinic
3       triclinic
4       triclinic
          ...    
144    monoclinic
145         cubic
146      trigonal
147     hexagonal
148         cubic
Name: crystal_system, Length: 149, dtype: object

In [14]:
X_test.shape

(149, 152)

In [47]:
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder

# Data Preprocessing
# Create a scaler object
scaler = StandardScaler()

# Fit scaler on the training data
scaler.fit(X_train)

# Apply the scaler to both the training and test data
X_train_normalized = scaler.transform(X_train)
X_test_normalized = scaler.transform(X_test)

# Convert pandas Series to NumPy array and reshape
y_train_array = y_train.to_numpy().reshape(-1, 1)
y_test_array = y_test.to_numpy().reshape(-1, 1)

# Initialize the OneHotEncoder
encoder = OneHotEncoder()

# Apply one-hot encoding
y_train_encoded = encoder.fit_transform(y_train_array).toarray()
y_test_encoded = encoder.transform(y_test_array).toarray()

import tensorflow as tf
from tensorflow import keras
from kerastuner.tuners import BayesianOptimization

def build_classification_model(hp):
    inputs = tf.keras.Input(shape=(152,))  # Update the input shape to match your dataset

    x = inputs

    # Add one or more dense layers with hyperparameter tuning
    for i in range(hp.Int("dense_layers", 1, 8, default=1)):
        x = tf.keras.layers.Dense(
            units=hp.Int('dense_units'+str(i), min_value=32, max_value=1024, step=32, default=32),
            activation='relu',
        )(x)
        x = tf.keras.layers.Dropout(hp.Float('dropout_dense_' + str(i), 0.1, 0.8, step=0.1, default=0.3))(x)

    # Output layer for classification with 7 classes
    outputs = tf.keras.layers.Dense(7, activation="softmax")(x)

    model = tf.keras.Model(inputs, outputs)

    model.compile(
        optimizer=keras.optimizers.Adam(hp.Choice('learning_rate', values=[1e-1, 1e-2, 1e-3, 1e-4])),
        loss='categorical_crossentropy',  # Use appropriate classification loss
        metrics=['accuracy']  # Use accuracy for classification
    )

    return model

# Define the BayesianOptimization tuner
tuner = BayesianOptimization(
    build_classification_model,
    objective='val_accuracy',
    num_initial_points=5,  # Number of random initializations
    max_trials=10,  # Number of Bayesian optimization iterations
    directory='tuner_dir',  # Directory to store results
    project_name='classification_model'
)

# Perform hyperparameter tuning
tuner.search(X_train_normalized, y_train_encoded, epochs=10, validation_split=0.2)

# Get the best hyperparameters
best_hparams = tuner.get_best_hyperparameters(num_trials=1)[0]

# Build the model with the best hyperparameters
best_model = build_classification_model(best_hparams)

# Compile the model
best_model.compile(
    optimizer=keras.optimizers.Adam(best_hparams.get('learning_rate')),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

# Train the model on the entire training dataset
history = best_model.fit(X_train_normalized, y_train_encoded, epochs=50)

# Evaluate the model on the training and testing data
train_accuracy = best_model.evaluate(X_train_normalized, y_train_encoded, verbose=0)[1]
test_accuracy = best_model.evaluate(X_test_normalized, y_test_encoded, verbose=0)[1]

# Print the results
print("Best Hyperparameters:")
print(best_hparams.get_config())
print("Training Accuracy:", train_accuracy)
print("Testing Accuracy:", test_accuracy)

Trial 10 Complete [00h 00m 02s]
val_accuracy: 0.5166666507720947

Best val_accuracy So Far: 0.5583333373069763
Total elapsed time: 00h 41m 38s
Epoch 1/50
19/19 [==============================] - 1s 5ms/step - loss: 4.1722 - accuracy: 0.5453
Epoch 2/50
19/19 [==============================] - 0s 5ms/step - loss: 2.0238 - accuracy: 0.5738
Epoch 3/50
19/19 [==============================] - 0s 5ms/step - loss: 1.9830 - accuracy: 0.5923
Epoch 4/50
19/19 [==============================] - 0s 5ms/step - loss: 1.8106 - accuracy: 0.5470
Epoch 5/50
19/19 [==============================] - 0s 5ms/step - loss: 2.9033 - accuracy: 0.5956
Epoch 6/50
19/19 [==============================] - 0s 5ms/step - loss: 4.3026 - accuracy: 0.5856
Epoch 7/50
19/19 [==============================] - 0s 5ms/step - loss: 5.1709 - accuracy: 0.5654
Epoch 8/50
19/19 [==============================] - 0s 5ms/step - loss: 3.3882 - accuracy: 0.5940
Epoch 9/50
19/19 [==============================] - 0s 6ms/step - loss: 4

In [48]:
df= pd.read_csv('Orbital_train.csv')
df.shape

(595, 1038)

In [51]:
X_train=df.iloc[:,14:]
y_train=df['crystal_system']

y_train

0          trigonal
1        tetragonal
2      orthorhombic
3      orthorhombic
4             cubic
           ...     
590    orthorhombic
591        trigonal
592      monoclinic
593       triclinic
594        trigonal
Name: crystal_system, Length: 595, dtype: object

In [52]:
X_train.shape

(595, 1024)

In [53]:
df=pd.read_csv('Orbital_test.csv')
X_test=df.iloc[:,14:]

y_test=df['crystal_system']

y_test

0           cubic
1           cubic
2       triclinic
3       triclinic
4       triclinic
          ...    
144    monoclinic
145      trigonal
146     hexagonal
147         cubic
148         cubic
Name: crystal_system, Length: 149, dtype: object

In [54]:
X_test.shape

(149, 1024)

In [55]:
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder

# Data Preprocessing
# Create a scaler object
scaler = StandardScaler()

# Fit scaler on the training data
scaler.fit(X_train)

# Apply the scaler to both the training and test data
X_train_normalized = scaler.transform(X_train)
X_test_normalized = scaler.transform(X_test)

# Convert pandas Series to NumPy array and reshape
y_train_array = y_train.to_numpy().reshape(-1, 1)
y_test_array = y_test.to_numpy().reshape(-1, 1)

# Initialize the OneHotEncoder
encoder = OneHotEncoder()

# Apply one-hot encoding
y_train_encoded = encoder.fit_transform(y_train_array).toarray()
y_test_encoded = encoder.transform(y_test_array).toarray()

import tensorflow as tf
from tensorflow import keras
from kerastuner.tuners import BayesianOptimization

def build_classification_model(hp):
    inputs = tf.keras.Input(shape=(1024,))  # Update the input shape to match your dataset

    x = inputs

    # Add one or more dense layers with hyperparameter tuning
    for i in range(hp.Int("dense_layers", 1, 8, default=1)):
        x = tf.keras.layers.Dense(
            units=hp.Int('dense_units'+str(i), min_value=32, max_value=1024, step=32, default=32),
            activation='relu',
        )(x)
        x = tf.keras.layers.Dropout(hp.Float('dropout_dense_' + str(i), 0.1, 0.8, step=0.1, default=0.3))(x)

    # Output layer for classification with 7 classes
    outputs = tf.keras.layers.Dense(7, activation="softmax")(x)

    model = tf.keras.Model(inputs, outputs)

    model.compile(
        optimizer=keras.optimizers.Adam(hp.Choice('learning_rate', values=[1e-1, 1e-2, 1e-3, 1e-4])),
        loss='categorical_crossentropy',  # Use appropriate classification loss
        metrics=['accuracy']  # Use accuracy for classification
    )

    return model

# Define the BayesianOptimization tuner
tuner = BayesianOptimization(
    build_classification_model,
    objective='val_accuracy',
    num_initial_points=5,  # Number of random initializations
    max_trials=10,  # Number of Bayesian optimization iterations
    directory='tuner_dir',  # Directory to store results
    project_name='classification_model'
)

# Perform hyperparameter tuning
tuner.search(X_train_normalized, y_train_encoded, epochs=10, validation_split=0.2)

# Get the best hyperparameters
best_hparams = tuner.get_best_hyperparameters(num_trials=1)[0]

# Build the model with the best hyperparameters
best_model = build_classification_model(best_hparams)

# Compile the model
best_model.compile(
    optimizer=keras.optimizers.Adam(best_hparams.get('learning_rate')),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

# Train the model on the entire training dataset
history = best_model.fit(X_train_normalized, y_train_encoded, epochs=50)

# Evaluate the model on the training and testing data
train_accuracy = best_model.evaluate(X_train_normalized, y_train_encoded, verbose=0)[1]
test_accuracy = best_model.evaluate(X_test_normalized, y_test_encoded, verbose=0)[1]

# Print the results
print("Best Hyperparameters:")
print(best_hparams.get_config())
print("Training Accuracy:", train_accuracy)
print("Testing Accuracy:", test_accuracy)

Reloading Tuner from tuner_dir/classification_model/tuner0.json
Epoch 1/50
19/19 [==============================] - 1s 5ms/step - loss: 1.6865 - accuracy: 0.5294
Epoch 2/50
19/19 [==============================] - 0s 5ms/step - loss: 1.2074 - accuracy: 0.5681
Epoch 3/50
19/19 [==============================] - 0s 5ms/step - loss: 1.0545 - accuracy: 0.6252
Epoch 4/50
19/19 [==============================] - 0s 6ms/step - loss: 1.0331 - accuracy: 0.6168
Epoch 5/50
19/19 [==============================] - 0s 6ms/step - loss: 1.0670 - accuracy: 0.6303
Epoch 6/50
19/19 [==============================] - 0s 5ms/step - loss: 1.0319 - accuracy: 0.6202
Epoch 7/50
19/19 [==============================] - 0s 5ms/step - loss: 1.0499 - accuracy: 0.6185
Epoch 8/50
19/19 [==============================] - 0s 5ms/step - loss: 1.1747 - accuracy: 0.6437
Epoch 9/50
19/19 [==============================] - 0s 5ms/step - loss: 1.2064 - accuracy: 0.6269
Epoch 10/50
19/19 [==============================] - 0

In [56]:
df= pd.read_csv('Xrd_train.csv')
df.shape

(596, 142)

In [57]:
X_train=df.iloc[:,14:]
y_train=df['crystal_system']

y_train

0          trigonal
1        tetragonal
2      orthorhombic
3      orthorhombic
4             cubic
           ...     
591      monoclinic
592        trigonal
593    orthorhombic
594        trigonal
595      monoclinic
Name: crystal_system, Length: 596, dtype: object

In [58]:
X_train.shape

(596, 128)

In [59]:
df=pd.read_csv('Xrd_test.csv')
X_test=df.iloc[:,14:]

y_test=df['crystal_system']

y_test

0          cubic
1          cubic
2      triclinic
3      triclinic
4      triclinic
         ...    
144     trigonal
145        cubic
146     trigonal
147    hexagonal
148        cubic
Name: crystal_system, Length: 149, dtype: object

In [60]:
X_test.shape

(149, 128)

In [61]:
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder

# Data Preprocessing
# Create a scaler object
scaler = StandardScaler()

# Fit scaler on the training data
scaler.fit(X_train)

# Apply the scaler to both the training and test data
X_train_normalized = scaler.transform(X_train)
X_test_normalized = scaler.transform(X_test)

# Convert pandas Series to NumPy array and reshape
y_train_array = y_train.to_numpy().reshape(-1, 1)
y_test_array = y_test.to_numpy().reshape(-1, 1)

# Initialize the OneHotEncoder
encoder = OneHotEncoder()

# Apply one-hot encoding
y_train_encoded = encoder.fit_transform(y_train_array).toarray()
y_test_encoded = encoder.transform(y_test_array).toarray()

import tensorflow as tf
from tensorflow import keras
from kerastuner.tuners import BayesianOptimization

def build_classification_model(hp):
    inputs = tf.keras.Input(shape=(128,))  # Update the input shape to match your dataset

    x = inputs

    # Add one or more dense layers with hyperparameter tuning
    for i in range(hp.Int("dense_layers", 1, 8, default=1)):
        x = tf.keras.layers.Dense(
            units=hp.Int('dense_units'+str(i), min_value=32, max_value=1024, step=32, default=32),
            activation='relu',
        )(x)
        x = tf.keras.layers.Dropout(hp.Float('dropout_dense_' + str(i), 0.1, 0.8, step=0.1, default=0.3))(x)

    # Output layer for classification with 7 classes
    outputs = tf.keras.layers.Dense(7, activation="softmax")(x)

    model = tf.keras.Model(inputs, outputs)

    model.compile(
        optimizer=keras.optimizers.Adam(hp.Choice('learning_rate', values=[1e-1, 1e-2, 1e-3, 1e-4])),
        loss='categorical_crossentropy',  # Use appropriate classification loss
        metrics=['accuracy']  # Use accuracy for classification
    )

    return model

# Define the BayesianOptimization tuner
tuner = BayesianOptimization(
    build_classification_model,
    objective='val_accuracy',
    num_initial_points=5,  # Number of random initializations
    max_trials=10,  # Number of Bayesian optimization iterations
    directory='tuner_dir',  # Directory to store results
    project_name='classification_model'
)

# Perform hyperparameter tuning
tuner.search(X_train_normalized, y_train_encoded, epochs=10, validation_split=0.2)

# Get the best hyperparameters
best_hparams = tuner.get_best_hyperparameters(num_trials=1)[0]

# Build the model with the best hyperparameters
best_model = build_classification_model(best_hparams)

# Compile the model
best_model.compile(
    optimizer=keras.optimizers.Adam(best_hparams.get('learning_rate')),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

# Train the model on the entire training dataset
history = best_model.fit(X_train_normalized, y_train_encoded, epochs=50)

# Evaluate the model on the training and testing data
train_accuracy = best_model.evaluate(X_train_normalized, y_train_encoded, verbose=0)[1]
test_accuracy = best_model.evaluate(X_test_normalized, y_test_encoded, verbose=0)[1]

# Print the results
print("Best Hyperparameters:")
print(best_hparams.get_config())
print("Training Accuracy:", train_accuracy)
print("Testing Accuracy:", test_accuracy)

Reloading Tuner from tuner_dir/classification_model/tuner0.json
Epoch 1/50
19/19 [==============================] - 1s 4ms/step - loss: 2.0126 - accuracy: 0.5503
Epoch 2/50
19/19 [==============================] - 0s 4ms/step - loss: 1.4960 - accuracy: 0.5973
Epoch 3/50
19/19 [==============================] - 0s 4ms/step - loss: 1.3019 - accuracy: 0.6258
Epoch 4/50
19/19 [==============================] - 0s 4ms/step - loss: 1.2503 - accuracy: 0.6326
Epoch 5/50
19/19 [==============================] - 0s 4ms/step - loss: 1.3219 - accuracy: 0.6275
Epoch 6/50
19/19 [==============================] - 0s 4ms/step - loss: 1.0961 - accuracy: 0.6477
Epoch 7/50
19/19 [==============================] - 0s 4ms/step - loss: 1.1099 - accuracy: 0.6594
Epoch 8/50
19/19 [==============================] - 0s 4ms/step - loss: 1.1988 - accuracy: 0.6309
Epoch 9/50
19/19 [==============================] - 0s 4ms/step - loss: 1.2114 - accuracy: 0.6577
Epoch 10/50
19/19 [==============================] - 0